# 0. Flow

- Web Scrapping
    - 사이트를 긁어서 데이터를 가져온다.
    - 웹에 접속한다
        - urllib.request.urlopen
        - get or post 방식(메소드)
    - 응답 페이지가 전송된다
        - 텍스트, html(반정형데이터 : 데이터와 구조가 같이 적용되어 있다.)
    - html 형태의 텍스트를 파싱한다.
        - html -> 메모리에 띄운다(DOM TREE)
            - DOM : Document Object Model
            - DOM 적용되면 모든 요소가 객체화 되어서 메모리에 로드된다.
            - **BS4** 담당
                - 파싱은 파서가 담당
                    - **html5lib : 느리지만 대용량 html도 잘 처리함!!** => 주로 사용
        - 필요한 데이터를 쿼리해서 추출한다
            - **css selector**, xpath 등 요소를 특정하는 규칙
                - css의 기술
            - **특정된 요소에서 데이터 추출 => BS4 담당**


- 웹
    - 프런트 / 클라이언트 사이드
        - 3개 요소로 구성
            - **html** : 화면을 구성하는 뼈대(골격), 콘텐츠(데이터)를 가지고 있다.
            - css : 화면의 레이아웃, 디자인, 프리젠테이션, 애니메이션, 반응형처리
                - 템플릿 활용
                    - bootstrap,
            - **javascript** : 사용자와 브라우저간 인터렉션 담당, 이벤트, 통신(ajax, 웹소켓), 화면 동적 구성
            - SPA(SIngle Page Application)
                - AngularJS (구글, 타입스크립트)
                - React JS (메타, 자바/타입스크립트)
                    - 웹, 모바일
                _ Vue (커뮤니티, 자바/타입스크립트)
    - 백엔드 / 서버 사이드
        - asp/php//servlet/jsp/ejb
        - spring/node
        - spring boot / node / .net(C#) / **Python(django, flask, fastapi)**
        - 조합
            - django + ReactJS
            - flask or fastapi => 모델 서빙
            - 속도 : django < flask < fastapi
    - 데이터베이스

# 1. 데이터 수집

- 타겟 사이트
    - https://finance.naver.com/marketindex/exchangeList.naver
- 특징
    - 하나은행 제공
    - 1일간 xxx회 갱신(고시회차)
- 목적
    - 타겟 사이트에 30초 간격(예시)으로 접속
    - 통화별 환율 정보, 현재 시간, 고시회차 획득
    - [{}...] -> DataFrame -> Database 입력
    - 자동화 스케줄
        - (월 ~ 금) 09시 ~ 18시까지만 진행한다.
        - 단, 휴일은 배제한다 => 무시 => 차후 결측치 처리

- 우선 자동화 전까지만 해보자!

## 1-1. Part1

- 타겟 사이트에 접속 -> 요청 -> 응답

In [ ]:
import urllib
import urllib.request as req
import json
# 타겟 사이트 정의
TARGET_SITE = 'https://finance.naver.com/marketindex/exchangeList.naver'
# 헤더정보 세팅같은 행위가 없으면 URL 문자열을 바로 넣어서 요청 처리
# 요청 및 응답수신
res = req.urlopen(TARGET_SITE)
res

## 1-2. Part2

- 수신한 html 텍스트를 파싱 후 데이터 추출

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# 파싱 -> html 텍스트 재료 -> 매모리상에 로드 (Dom Tree를 구성한다)
soup = BeautifulSoup(res, 'html5lib') # soup라는 변수명을 자주 사용한다,
print(soup)

<html lang="ko"><head>
<title>네이버 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230808201105/css/finance.css" rel="stylesheet" type="text/css"/>

<script language="javascript">document.domain="naver.com";</script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20230808201105/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20230808201105/js/lcslog.js" type="text/javascript"></script>
</head>
<body>
<div class="tbl_area">
	<table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
	<caption>환전 고시 환율</caption>
	<colgroup>
		<col width="162"/>
		<col width="92"/>
		<col width="92"/>
		<col width="92"/>
		<col width="93"/>
		<col width="92"/>
		<col width="90"/>
	</colgroup>
	<thead>
	<tr

# 2. 데이터 추출

In [ ]:
# 매매 기준율 -> .sale
for sale_std_rate in soup.select('.sale') :
    print(sale_std_rate.text)

1,312.50
1,441.65
915.75
181.87
167.89
41.36
1,672.98
3,409.09
978.60
1,499.31
123.17
860.34
797.28
59.34
1.52
48.56
0.38
353.05
193.48
128.83
349.82
4,267.88
3,481.62
357.34
1,851.20
42.41
37.59
976.53
287.14
8.66
360.18
2.94
976.53
15.85
4.57
11.99
23.32
76.85
267.81
5.53
69.24
13.50
3.72
323.32
4.11
9.66
9.15
0.32
0.52
9.90
291.38
273.93
162.98
0.63
23.80
0.11
0.32
586.88


In [ ]:
for sale_std_rate in soup.select('.tit') :
    print(sale_std_rate.text.strip(), end=', ')
    break

미국 USD, 

In [ ]:
# 요구사항
# 통화명을 출력하시오 class = 'tit'
# 통화명, 통화명, ... 으로 나열
for sale_std_rate in soup.select('.tit>a') :
    print(sale_std_rate.text.strip(), end=', ')

미국 USD, 유럽연합 EUR, 일본 JPY (100엔), 중국 CNY, 홍콩 HKD, 대만 TWD, 영국 GBP, 오만 OMR, 캐나다 CAD, 스위스 CHF, 스웨덴 SEK, 호주 AUD, 뉴질랜드 NZD, 체코 CZK, 칠레 CLP, 튀르키예 TRY, 몽골 MNT, 이스라엘 ILS, 덴마크 DKK, 노르웨이 NOK, 사우디아라비아 SAR, 쿠웨이트 KWD, 바레인 BHD, 아랍에미리트 AED, 요르단 JOD, 이집트 EGP, 태국 THB, 싱가포르 SGD, 말레이시아 MYR, 인도네시아 IDR 100, 카타르 QAR, 카자흐스탄 KZT, 브루나이 BND, 인도 INR, 파키스탄 PKR, 방글라데시 BDT, 필리핀 PHP, 멕시코 MXN, 브라질 BRL, 베트남 VND 100, 남아프리카 공화국 ZAR, 러시아 RUB, 헝가리 HUF, 폴란드 PLN, 스리랑카 LKR, 알제리 DZD, 케냐 KES, 콜롬비아 COP, 탄자니아 TZS, 네팔 NPR, 루마니아 RON, 리비아 LYD, 마카오 MOP, 미얀마 MMK, 에티오피아 ETB, 우즈베키스탄 UZS, 캄보디아 KHR, 피지 FJD, 

In [ ]:
# 요구사항
# "통화별(58개) 현찰항목의 하위 항목 사실때"의 데이터를 모두 추출하시오
# 상위레벨에서 반복하여 하위를 추출한다(패턴)
# 결과 : 수치, 수치, 수치, ...

# 통화별로 한 덩어리 html 객체 추출
len( soup.select('.tbl_exchange>tbody>tr') )
for tr in soup.select('.tbl_exchange>tbody>tr'):
    # 한 덩어리 내에서 다시 탐색해서 세부 정보 추출
    print(tr.select_one('td:nth-of-type(3)').text.strip().replace(',',''), end=', ')

1335.46, 1470.33, 931.77, 190.96, 171.19, 46.77, 1705.93, 3647.72, 997.87, 1528.84, 127.23, 877.28, 812.98, 64.38, 1.67, N/A, N/A, 388.35, 199.86, 133.08, 371.85, 4545.29, 3704.44, 376.99, 2015.95, N/A, 39.46, 995.96, 308.38, 9.52, N/A, N/A, 1015.59, N/A, N/A, N/A, 25.65, 84.91, 295.12, 6.18, 74.77, 14.78, 4.06, 349.18, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, 

In [ ]:
# 전체 후보군(406개)을 다 뽑아서 거기서 돌아가면서 솎아서 추출한다? => 비효율성이 보임
for i, td in enumerate( soup.select('.tbl_exchange>tbody td')):
    # 2, 9, 16, ..
    # 2+(7*0), 2+(7*1), 2+(7*2), ...
    #if 조건식:
    #    print( 출력, end=', ' )
    pass

# 3. 함수화

In [ ]:
# 요구사항
'''
[
    {
       'name':'미국 USD'
       'std_rate' : 1320.55,
       'cash_buy' : 1343.55
       'rate' : 1,000
       'code' : 'USD'
    },
    {
        ...
    },
    ...


]
'''

"\n[\n    {\n       'name':'미국 USD'\n       'std_rate' : 1320.55,\n       'cash_buy' : 1343.55\n       'rate' : 1,000\n       'code' : 'USD'\n    },\n    {\n        ...\n    },\n    ...\n\n\n]\n"

In [ ]:
# name 추출
for title in soup.select('.tit') :
    print(title.text.strip())
    break

미국 USD


In [ ]:
# 일반 함수
def my_func2(tr, selector) :
    return tr.select_one(selector).text.strip()
# 람다 함수
my_func = lambda tr, selector : tr.select_one(selector).text.strip()
my_func3 = lambda tr, selector : tr.select_one(selector).text.strip().replace(',','').replace('N/A', '0.0')

exchange = [
    {
        'name'       : my_func(tr, '.tit>a'),
        'std_rate'   : my_func(tr, '.sale' ),
        'cash_buy'   : my_func3(tr, 'td:nth-of-type(3)').replace(',',''),
        'rate'       : my_func3(tr, 'td:nth-of-type(7)').replace(',',''),
        'code'       : tr.select_one('.tit>a').get('href')[-6:-3]
    }
    for tr in soup.select('.tbl_exchange>tbody>tr')
]
# 추출된 값을 체크해 보니 N/A가 발견되었다.-> '0.0'으로 대체

In [ ]:
# 한칸에서 모두 완결
# 시간정보, 공급자, 회차 (위의 실습과 다른 페이지임)
# 타겟 사이트 : https://finance.naver.com/marketindex/?tabSel=exchange#tab_section

TARGET_SITE = 'https://finance.naver.com/marketindex/?tabSel=exchange#tab_section'
res = req.urlopen(TARGET_SITE)
soup = BeautifulSoup(res, 'html5lib') # soup라는 변수명을 자주 사용한다,
soup.select('.date')

[<span class="date">2023.08.09 13:50</span>]

In [ ]:
print(soup.select_one('.date').text.strip())
print(soup.select_one('.standard').text.strip())
print(soup.select_one('.round').text.strip())

2023.08.09 13:50
하나은행 기준
고시회차 452회


# 4. 최종 통합

In [ ]:
def get_exchange_info():
    #요청 -> 응답
    res = req.urlopen('https://finance.naver.com/marketindex/?tabSel=exchange#tab_section')
    #파싱 -> soup
    soup = BeautifulSoup(res, 'html5lib')
    #탐색 및 추출
    date = soup.select_one('.date').text.strip()
    bank = soup.select_one('.standard').text.strip().split('은행')[0]
    round = soup.select_one('.round>em').text.strip()
    #출력 값 생성
    return {
        'date' : date,
        'bank' : bank,
        'round': round
        }

print(get_exchange_info())
public_info = get_exchange_info()

{'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}


In [ ]:
def my_func2( tr, selector):
    return tr.select_one(selector).text.strip()

my_func = lambda tr, selector:tr.select_one(selector).text.strip()
my_func_ex = lambda tr, selector:tr.select_one(selector).text.strip().replace(',','').replace('N/A','0.0')
exchange = [
    {
        'name'      :my_func(tr, '.tit > a'),
        'std_rate'  :my_func(tr, '.sale'),
        'cash_buy'  :my_func_ex(tr, 'td:nth-of-type(3)'),
        'rate'      :my_func_ex(tr, 'td:nth-of-type(7)'),
        'code'      :tr.select_one('.tit > a').get('href')[-6:-6+3]
        # date, bank, round 추가
        ,
        'date'  : public_info['date'],
        'bank'  : public_info['bank'],
        'round' : public_info['round']
    }
    for tr in soup.select('.tbl_exchange>tbody>tr')#[:3]
]
# 추출된 값을 체크해 보니 N/A가 발견되었다 -> '0.0'으로 대체

In [ ]:
print(exchange)

[{'name': '미국 USD', 'std_rate': '1,317.20', 'cash_buy': '1340.25', 'rate': '1.000', 'code': 'USD', 'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}, {'name': '유럽연합 EUR', 'std_rate': '1,445.76', 'cash_buy': '1474.53', 'rate': '1.098', 'code': 'EUR', 'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}, {'name': '일본 JPY (100엔)', 'std_rate': '920.12', 'cash_buy': '936.22', 'rate': '0.699', 'code': 'JPY', 'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}, {'name': '중국 CNY', 'std_rate': '182.46', 'cash_buy': '191.58', 'rate': '0.139', 'code': 'CNY', 'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}, {'name': '홍콩 HKD', 'std_rate': '168.54', 'cash_buy': '171.86', 'rate': '0.128', 'code': 'HKD', 'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}, {'name': '대만 TWD', 'std_rate': '41.42', 'cash_buy': '46.84', 'rate': '0.031', 'code': 'TWD', 'date': '2023.08.09 13:50', 'bank': '하나', 'round': '452'}, {'name': '영국 GBP', 'std_rate': '1,681.41', 'cash_buy': '1714.5